In [ ]:
import pandas as pd
import numpy as np
import os
import math
import statistics as stats
import gurobipy as gb
from gurobipy import *

### Read in Data

In [ ]:
# set to your working directory
dir = r"C:\Users\tnaga\Documents\MMA\Fall 2022\Decision Analytics\Project\Test"
os.chdir(dir)
dft = pd.read_excel("Model_Data.xlsx", sheet_name="Transitional")
dfe = pd.read_excel("Model_Data.xlsx", sheet_name="Emergency")
dfh = pd.read_excel("Model_Data.xlsx", sheet_name="HotSpots")
dfn = pd.read_excel("Model_Data.xlsx", sheet_name="NewLocations")

### Define Haversine Distance Calculation

In [ ]:
def haversine_distance(coord1: list, coord2: list):
    # calculates the angular distance along a sphere between latitude and longitude coordinates
    lon1, lat1 = coord1
    lon2, lat2 = coord2

    R = 6371000
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c 
    km = meters / 1000.0

    meters = round(meters, 3)
    km = round(km, 3)
    miles = km * 0.621371
    return km

### Data Preparation

In [ ]:
# 50% of homeless people assigned to metros
# 50% of homeless people assigned to homeless camps
pct = 0.5

# total number of homeless people
total_homeless = 3573
total_metro = math.floor(total_homeless * pct)
total_camp = total_homeless - total_metro

### Transitional

In [ ]:
# Calculate Total Number of beds
dft_piv = pd.pivot_table(dft, values="Beds", index="Index", aggfunc=np.sum)
dft_piv.columns = ["Total Beds"]
dft = pd.merge(dft, dft_piv, how = "left", left_on="Index", right_on="Index")

# Replace missing values with median value of costs
dft['Annual Cost'] = dft['Annual Cost'].replace({np.nan:dft['Annual Cost'].median()})

# calculate adjusted costs based on number of beds
dft['Adjusted Costs'] = (dft['Beds'] / dft['Total Beds']) * dft['Annual Cost']

# calculate the variable cost
var_cost = 25860
dft['Variable Cost'] = dft['Beds'] * var_cost

dft = dft.drop(columns="Index")

### Emergency

In [ ]:
# Calculate Total Number of Beds
dfe_piv = pd.pivot_table(dfe, values="Beds", index="Index", aggfunc=np.sum)
dfe_piv.columns = ["Total Beds"]
dfe = pd.merge(dfe, dfe_piv, how = "left", left_on="Index", right_on="Index")

# replace missing values with median value of costs
dfe['Annual Cost'] = dfe['Annual Cost'].replace({np.nan:dfe['Annual Cost'].median()})

# calculated adjusted costs based on number of beds
dfe['Adjusted Costs'] = (dfe['Beds'] / dfe['Total Beds']) * dfe['Annual Cost']

# calculate the variable cost
var_cost = 25860
dfe['Variable Cost'] = dfe['Beds'] * var_cost

dfe = dfe.drop(columns="Index")

### Combine Transitional and Emergency

In [ ]:
dfc = pd.concat([dft, dfe])

### Hotspots

In [ ]:
dfh_camp = dfh[pd.isna(dfh['Density'])]
dfh_camp = dfh_camp[['Station', 'Density', 'Latitude', 'Longitude']].reset_index()
dfh_metro = dfh[pd.notna(dfh['Density'])]
dfh_metro = dfh_metro[['Station', 'Density', 'Latitude', 'Longitude']].reset_index()

In [ ]:
# distribute homeless in camps equally except West Island
num_camps = dfh_camp.shape[0]
dfh_camp['Homeless'] = (total_camp - 25) // (num_camps - 1)
dfh_camp.loc[dfh_camp['Station'] == "West Island",'Homeless'] = 25 # Set West Island to 25

dfh_camp['Male'] = dfh_camp['Homeless'] * 0.7
dfh_camp['Male'] = dfh_camp['Male'].round() + 1

dfh_camp['Female'] = dfh_camp['Homeless'] * 0.3
dfh_camp['Female'] = dfh_camp['Female'].round()

dfh_camp.loc[dfh_camp['Station'] == "West Island",'Male'] = 18
dfh_camp.loc[dfh_camp['Station'] == "West Island",'Female'] = 7

dfh_camp = dfh_camp.drop(columns=['Density'])

In [ ]:
# find the minimum density value
# put density of metro station divided by minimum density to create a scale
low = dfh_metro['Density'].min()
dfh_metro['Scale'] = dfh_metro['Density'] / low
total = dfh_metro['Scale'].sum()

# calculate homeless distribution by taking scale divided by total scale multiplied by total number of homeless assigned to metros
dfh_metro['Homeless'] = dfh_metro['Scale'] / total * total_metro
dfh_metro['Homeless'] = dfh_metro['Homeless'].round()

# determine portion male and portion female
dfh_metro['Male'] = dfh_metro['Homeless'] * 0.7
dfh_metro['Male'] = dfh_metro['Male'].round()

dfh_metro['Female'] = dfh_metro['Homeless'] * 0.3
dfh_metro['Female'] = dfh_metro['Female'].round()

dfh_metro = dfh_metro.drop(columns=['Density', 'Scale'])

In [ ]:
# Concatenate metro and camps
dfd = pd.concat([dfh_metro, dfh_camp]).reset_index()

### Potential

In [ ]:
dfn.head()

### Export to Excel

In [ ]:
with pd.ExcelWriter('Final_Model_Data.xlsx') as writer:
    dft.to_excel(writer, sheet_name="Transitional", index=False)
    dfe.to_excel(writer, sheet_name="Emergency", index=False)
    dfc.to_excel(writer, sheet_name="CombinedHousing", index=False)
    dfh_camp.to_excel(writer, sheet_name="HotspotCamps", index=False)
    dfh_metro.to_excel(writer, sheet_name="HotspotsMetro", index=False)
    dfd.to_excel(writer, sheet_name="HotspotsAll", index=False)
    dfn.to_excel(writer, sheet_name="NewLocations", index=False)

### Model 1: Simulation

#### Model 1 Data Prep

In [ ]:
hotspot = dfd.copy()
shelter = dfc.copy().reset_index()

In [ ]:
shelter.sort_values(by='Code',inplace=True) # sort by Code to segment All, Female and Male shelters
shelter.reset_index(inplace=True)
shelter.drop(columns='index',inplace=True)

In [ ]:
Shelter = shelter.loc[:,'Beds'].tolist()
Females = hotspot.loc[:,'Female'].tolist()
Males = hotspot.loc[:,'Male'].tolist()
Gender = ['Female', 'Male']

In [ ]:
H = len(hotspot) # number of hotspots
S = len(shelter) # number of shelters
G = len(Gender) # number of genders

In [ ]:
# calculate the distances between all hotspots and existing shelters
Distance=[]
for i in range(H):
    l=[]
    hotspot_coord=[hotspot.loc[i,'Latitude'],hotspot.loc[i,'Longitude']]
    for j in range(S):
        shelter_coord=[shelter.loc[j,'Latitude'],shelter.loc[j,'Longitude']]
        
        dis=haversine_distance(hotspot_coord,shelter_coord)
        l.append(dis)
    Distance.append(l)

In [ ]:
A = shelter[shelter['Code']==0].shape[0] # Number of shelters open to all
F = shelter[shelter['Code']==1].shape[0] # Number of shelters open to only females
M = shelter[shelter['Code']==2].shape[0] # Number of shelters open to only males

#### Add Decision Variables and Set Objective Function

In [ ]:
prob = gb.Model("Shelters")

X = prob.addVars(H, S, G, lb=0, vtype=GRB.INTEGER, name=['X_Hotspot_'+str(i)+'_toShelter_'+str(j)+'_'+k for i in range(1,H+1) for j in range(1,S+1) for k in Gender])

prob.setObjective(gb.quicksum(X[i,j,k] for i in range(H) for j in range(S) for k in range(G)),GRB.MAXIMIZE)

#### Implement Constraints

In [ ]:
# Homeless People Constraints
# Female shelters can only have Females
for j in range(A,A+F):
    prob.addConstr(sum(X[i,j,0] for i in range(H))<=Shelter[j])

# Stop Males from going to Female Shelters
for j in range(A,A+F):
    prob.addConstr(sum(X[i,j,1] for i in range(H))==0)  

# Male shelters can only have Males
for j in range(A+F,A+F+M):
    prob.addConstr(sum(X[i,j,1] for i in range(H))<=Shelter[j])

# Stop Females from going to Male Shelters
for j in range(A+F,A+F+M):
    prob.addConstr(sum(X[i,j,0] for i in range(H))==0)

# Males from hotspots less than Males in Hotspots
for i in range(H):
    prob.addConstr(sum(X[i,j,1] for j in range(S))<=Males[i])

#Females from hotspots less than Females in Hotspots
for i in range(H):
    prob.addConstr(sum(X[i,j,0] for j in range(S))<=Females[i])

# Total from hotspots less than total in Hotspots
for j in range(0,A):
    prob.addConstr(sum(X[i,j,k] for i in range(H) for k in range(G))<=Shelter[j])

In [ ]:
# Distance constraints between hotspots and shelters set to maximum 4km
for i in range(H):
    for j in range(S):
        for k in range(G):
            prob.addConstr(X[i,j,k]*(4-Distance[i][j])>=0)

In [ ]:
prob.optimize()

In [ ]:
print(f'The total number of people in shelters : {prob.ObjVal}')
print(f'The optimal allocation is ')
for i in prob.getVars():
        print(f"{i.VarName}: {i.X}")

In [ ]:
# calculate the number remaining at each hotspot from model 1 as input to model 2
rem_male=[]
rem_female=[]
for i in range(H):
    f=0
    m=0
    for j in range(S):
        f = f + X[i,j,0].x 
        m = m + X[i,j,1].x
    rem_male.append(m)
    rem_female.append(f)

hotspot['Remaining']=hotspot['Homeless']-np.array(rem_male)-np.array(rem_female)
hotspot['Female_Rem']=hotspot['Female']-np.array(rem_female)
hotspot['Male_Rem']=hotspot['Male']-np.array(rem_male)

In [ ]:
# calculate the number of beds unoccupied at each shelter
unocc=[]
for i in range(S):
    s=0
    for j in range(H):
        s = s + X[j,i,0].x + X[j,i,1].x
    unocc.append(s)

shelter['Remaining']=shelter['Beds']-np.array(unocc)

In [ ]:
shelter[shelter['Remaining']>0]

### Model 2: New Shelter Allocation

#### Data Prep

In [ ]:
# use hotspot data from previous Model
potential = dfn.copy().reset_index()
shelter = dfc.copy().reset_index()
shelter.sort_values(by='Code',inplace=True) # sort by Code to segment All, Female and Male shelters
shelter.reset_index(inplace=True)
shelter.drop(columns='index',inplace=True)

In [ ]:
leftover=hotspot[hotspot['Remaining']>0].copy()
leftover['Annual_Cost_Homeless']=25860
leftover.drop(columns=['level_0','index'],inplace=True)
leftover.reset_index(inplace=True)
leftover.drop(columns='index',inplace=True)

In [ ]:
P = len(potential) # Number of Potential Shelters
S = len(shelter) # Number of Existing Shelters
L = len(leftover) # Number of Hotspots with Unallocated Homeless People

In [ ]:
# Calculate the fixed cost of new shelters by taking average cost of 3 closest shelters in original data set
dummy=pd.DataFrame({'Location':[],'Shelter':[],'Distance':[],'Adjusted Cost':[]})

# calculate the distance between all potential shelters and existing shelters
for i in range(P):
    potential_coord=[potential.loc[i,'Latitude'],potential.loc[i,'Longitude']]
    for j in range(S):
        shelter_coord=[shelter.loc[j,'Latitude'],shelter.loc[j,'Longitude']]
        
        dis=haversine_distance(potential_coord,shelter_coord)
        dummy=pd.concat([dummy,pd.DataFrame({'Location':potential.loc[i,'Location'],'Shelter':shelter.loc[j,'Name'],
                      'Distance':dis,'Adjusted Cost':shelter.loc[j,'Adjusted Costs']},index=[0])])

dummy.reset_index(inplace=True)
dummy.drop(columns='index',inplace=True)


In [ ]:
# average the adjusted cost of the 3 closest shelters to get the potential shelter adjusted costs
dummy['row_num']=dummy.groupby(by='Location')['Distance'].rank(method='first')
dummy=dummy[dummy['row_num'].isin([1,2,3])]
dum=pd.DataFrame(dummy.groupby(by='Location')['Adjusted Cost'].mean())
potential=potential.join(dum,how='inner',on='Location')

In [ ]:
# Calculate the distance between all potential shelters and hotspots with unallocated individuals
Distance=[]
for i in range(L):
    l=[]
    leftover_coord=[leftover.loc[i,'Latitude'],leftover.loc[i,'Longitude']]
    for j in range(P):
        potential_coord=[potential.loc[j,'Latitude'],potential.loc[j,'Longitude']]
        
        dis=haversine_distance(leftover_coord,potential_coord)
        l.append(dis)
    Distance.append(l)

In [ ]:
Shelters = potential.loc[:,'Capacity'].tolist()
Homeless = leftover.loc[:,'Remaining'].tolist()
Fixed_Cost = potential.loc[:,'Adjusted Cost'].tolist()
Cost_Per_Homeless = leftover.loc[:,'Annual_Cost_Homeless'].tolist()

#### Create Decision Variables and Set Objective Function

In [ ]:
prob1 = gb.Model("Shelter model 2")

X = prob1.addVars(L, P, lb=0, vtype=GRB.INTEGER, name=['X_LOC_'+str(i+1)+'_toShelter_'+str(j+1) for i in range(L) for j in range(P)])
D = prob1.addVars(P, vtype=GRB.BINARY)

prob1.setObjective(sum(Cost_Per_Homeless[i]*X[i,j] for i in range(L) for j in range(P))+sum(Fixed_Cost[i]*D[i] for i in range(P)),GRB.MINIMIZE)

# Add Constraints to Model

In [ ]:
prob1.addConstr(sum(D[i] for i in range(P))<=7)  #only one of the three locations will be picked

for i in range(L):
    for j in range(P):
        prob1.addConstr(X[i,j]*(4-Distance[i][j])>=0) #distance constraint
        
#sum of homeless allocated has to be less than or equal to the capacity
for j in range(P):
    prob1.addConstr(sum(X[i,j] for i in range(L))<=Shelters[j])

# sum of homeless allocated has to be less than or equal to the number of homeless remaining at hotspots
for i in range(L):
    prob1.addConstr(sum(X[i,j] for j in range(P))<=Homeless[i])
    
# all homeless have to be allocated to a shelter
prob1.addConstr(sum(X[i,j] for i in range(L) for j in range(P))>= 0.90*sum(Homeless[i] for i in range(L)))

# a shelter can only host homeless if it is chosen
M=1000000
for i in range(L):
     for j in range(P):
        prob1.addConstr(X[i,j]<=D[j]*M)

prob1.optimize()

In [ ]:
for v in prob1.getVars():
    if v.x>0:
        print(v.varName, "=", v.x)

In [ ]:
t=[]

for i in range(len(leftover)):
    s=0
    for j in range(len(potential)):
        s=s+X[i,j].x
    t.append(s)
print(f'The total people newly assigned to shelters are {sum(t)}')

In [ ]:
leftover['Remaining 1']=leftover['Remaining']-np.array(t)
leftover[leftover['Remaining 1'] > 0]

In [ ]:
print('The total people still unallocated : ',sum(leftover['Remaining 1']))

In [ ]:
print('The total optimal cost : ',prob1.objVal)